In [10]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_img_gen = ImageDataGenerator(rescale=1./255, validation_split = 0.25)
test_img_gen = ImageDataGenerator(rescale=1./255)


Training_Gen = train_img_gen.flow_from_directory('Eye_Images/Train', class_mode='binary', color_mode='grayscale', 
                                batch_size=32, target_size=(66,66), seed=711, 
                                subset='training')

Validation_Gen = train_img_gen.flow_from_directory('Eye_Images/Train', class_mode='binary', color_mode='grayscale', 
                                batch_size=32, target_size=(66,66), seed=711, 
                                subset='validation')

Testing_Gen=test_img_gen.flow_from_directory('Eye_Images/Test', class_mode='binary', color_mode='grayscale', 
                                batch_size=32, target_size=(66,66), seed=711)


Found 50940 images belonging to 2 classes.
Found 16979 images belonging to 2 classes.
Found 16979 images belonging to 2 classes.


In [12]:
training = image_dataset_from_directory('Eye_Images', labels='inferred', label_mode='binary', color_mode='grayscale', 
                                        batch_size=32, image_size=(66,66), validation_split = 0.2, subset='training', seed=711)

test = image_dataset_from_directory('Eye_Images', labels='inferred', label_mode='binary', color_mode='grayscale', 
                                        batch_size=32, image_size=(66,66), validation_split = 0.2, subset='validation', seed=711)

Found 84898 files belonging to 2 classes.
Using 67919 files for training.
Found 84898 files belonging to 2 classes.
Using 16979 files for validation.


In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, InputLayer, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [14]:
def create_model():    
    model = Sequential()
    model.add(InputLayer(input_shape=(66,66,1)))
    model.add(Conv2D(filters = 32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(units = 1, activation='sigmoid')) #sigmoid instead of softmax because binary output
    
    model.compile(optimizer='sgd', loss= 'binary_crossentropy', metrics='BinaryAccuracy')
    #model.load_weights(checkpoint_path)

    
    return model

In [15]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [18]:
model.save('full_batch_model_v2.h5')

In [16]:
model_output= model.fit(Training_Gen, epochs=50, validation_data=Validation_Gen,
                     callbacks=[ReduceLROnPlateau(factor=.5, patience=3, verbose=1)])